<a href="https://colab.research.google.com/github/fleithpi/PROGRAMA-O-ORIENTADA-A-OBJETOS/blob/main/REVISTA_com_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from datetime import date, datetime, timedelta # IMPORTAÇÕES DE DATA
from typing import Union
import sys

# Definindo o tipo unificado para o acervo
ItemAcervo = Union['Livro', 'Revista']


# --- 1. CLASSE LIVRO (Sem alterações relevantes) ---
class Livro:
    def __init__(self, titulo: str, autor: str):
        self.__titulo = titulo.strip()
        self.__autor = autor.strip()
        self.__disponivel = True

    @property
    def titulo(self) -> str: return self.__titulo
    @property
    def autor(self) -> str: return self.__autor
    @property
    def disponivel(self) -> bool: return self.__disponivel

    def emprestar(self) -> bool:
        if self.__disponivel:
            self.__disponivel = False
            return True
        return False

    def devolver(self) -> bool:
        if not self.__disponivel:
            self.__disponivel = True
            return True
        return False

    def __str__(self):
        status = "Disponível" if self.disponivel else "Emprestado"
        return f"[LIVRO] '{self.titulo}' por {self.autor} | Status: {status}"


# --- 2. CLASSE REVISTA (Sem alterações relevantes) ---
class Revista:
    def __init__(self, titulo: str, edicao: str, autor_editora: str):
        self.__titulo = titulo.strip()
        self.__edicao = edicao.strip()
        self.__autor = autor_editora.strip()
        self.__disponivel = True

    @property
    def titulo(self) -> str: return self.__titulo
    @property
    def autor(self) -> str: return self.__autor
    @property
    def edicao(self) -> str: return self.__edicao
    @property
    def disponivel(self) -> bool: return self.__disponivel

    def emprestar(self) -> bool:
        if self.__disponivel:
            self.__disponivel = False
            return True
        return False

    def devolver(self) -> bool:
        if not self.__disponivel:
            self.__disponivel = True
            return True
        return False

    def __str__(self):
        status = "Disponível" if self.disponivel else "Emprestada"
        return f"[REVISTA] '{self.titulo}' (Ed. {self.edicao}) | Editora: {self.autor} | Status: {status}"


# --- 3. CLASSE USUARIO (Sem alterações) ---
class Usuario:
    def __init__(self, id_usuario: int, nome: str):
        self.__id = id_usuario
        self.__nome = nome.strip()

    @property
    def id(self) -> int: return self.__id
    @property
    def nome(self) -> str: return self.__nome

    def __str__(self): return f"Usuário: {self.nome} (ID: {self.id})"


# --- 4. CLASSE BIBLIOTECA (ATUALIZADA COM DATAS) ---
class Biblioteca:
    def __init__(self, dias_max_emprestimos: int = 7): # NOVO: Prazo máximo
        self.__acervo: dict[str, ItemAcervo] = {}
        self.__usuarios: dict[int, Usuario] = {}
        self.__prazo_dias = dias_max_emprestimos # NOVO: Duração do prazo

        # ATUALIZADO: Valor é a tupla (ID do Usuário, Data Limite de Devolução)
        self.__emprestimos: dict[str, tuple[int, date]] = {}

    # --- MÉTODOS DE MANUTENÇÃO (Sem alterações relevantes) ---

    def adicionar_item(self, item: ItemAcervo):
        titulo = item.titulo
        tipo = "Revista" if isinstance(item, Revista) else "Livro"
        if titulo in self.__acervo:
            print(f"ATENÇÃO: Item '{titulo}' já está no acervo.")
        else:
            self.__acervo[titulo] = item
            print(f"SUCESSO: {tipo} '{titulo}' adicionado ao acervo.")

    def adicionar_usuario(self, usuario: Usuario):
        user_id = usuario.id
        if user_id in self.__usuarios:
            print(f"ATENÇÃO: Usuário ID {user_id} já está registrado.")
        else:
            self.__usuarios[user_id] = usuario
            print(f"SUCESSO: Usuário '{usuario.nome}' (ID: {user_id}) registrado.")

    def buscar_usuario(self, id_usuario: int) -> Usuario | None:
        return self.__usuarios.get(id_usuario)

    def buscar_item(self, titulo_item: str) -> ItemAcervo | None:
        return self.__acervo.get(titulo_item.strip())

    # --- MÉTODOS DE TRANSAÇÃO (ATUALIZADOS COM DATA) ---

    def emprestar_item(self, id_usuario: int, titulo_item: str):
        """Gerencia o empréstimo de Livro ou Revista e registra a data de devolução."""
        titulo_item = titulo_item.strip()

        usuario = self.buscar_usuario(id_usuario)
        if not usuario:
            print(f"ERRO: Usuário ID {id_usuario} não encontrado.")
            return

        item = self.buscar_item(titulo_item)
        if not item:
            print(f"ERRO: Item '{titulo_item}' não encontrado no acervo.")
            return

        tipo = "Revista" if isinstance(item, Revista) else "Livro"

        if item.emprestar():
            hoje = datetime.now().date()
            data_devolucao = hoje + timedelta(days=self.__prazo_dias) # CÁLCULO DA DATA LIMITE

            self.__emprestimos[titulo_item] = (id_usuario, data_devolucao) # ARMAZENAMENTO DA DATA

            print(f"EMPRÉSTIMO SUCESSO: {tipo} '{item.titulo}' emprestado para {usuario.nome}.")
            print(f"  -> DEVOLUÇÃO LIMITE: {data_devolucao.strftime('%d/%m/%Y')}") # EXIBE A DATA
        else:
            id_quem_emprestou, data_limite = self.__emprestimos.get(titulo_item, (None, None))
            nome_quem_emprestou = self.buscar_usuario(id_quem_emprestou).nome if id_quem_emprestou else "Desconhecido"
            data_str = data_limite.strftime('%d/%m/%Y') if data_limite else "N/A"
            print(f"EMPRÉSTIMO FALHA: {tipo} '{item.titulo}' já está emprestado para {nome_quem_emprestou} (Limite: {data_str}).")


    def devolver_item(self, id_usuario: int, titulo_item: str):
        """Gerencia a devolução de Livro ou Revista e verifica atraso."""
        titulo_item = titulo_item.strip()

        item = self.buscar_item(titulo_item)
        if not item:
            print(f"ERRO: Item '{titulo_item}' não encontrado para devolução.")
            return

        tipo = "Revista" if isinstance(item, Revista) else "Livro"
        usuario_nome = self.buscar_usuario(id_usuario).nome if self.buscar_usuario(id_usuario) else "Usuário Desconhecido"

        if item.devolver():
            if titulo_item in self.__emprestimos:
                id_quem_pegou, data_devolucao_limite = self.__emprestimos.pop(titulo_item) # Pega a data antes de remover

                hoje = datetime.now().date()
                if hoje > data_devolucao_limite:
                    dias_atraso = (hoje - data_devolucao_limite).days
                    print(f"DEVOLUÇÃO COM ATRASO: {tipo} '{item.titulo}' devolvida por {usuario_nome}.")
                    print(f"  -> ATRASO: {dias_atraso} dias.")
                else:
                    print(f"DEVOLUÇÃO SUCESSO: {tipo} '{item.titulo}' devolvida à biblioteca por {usuario_nome}.")
            else:
                 print(f"DEVOLUÇÃO SUCESSO: {tipo} '{item.titulo}' devolvida à biblioteca (Rastreio de empréstimo não encontrado).")
        else:
            print(f"DEVOLUÇÃO FALHA: {tipo} '{item.titulo}' já estava disponível. Verifique o título.")

    # --- Método de Visualização Simples (ATUALIZADO COM DATA) ---
    def listar_acervo(self):
        """Lista o estado atual de todos os itens (Livros e Revistas) no acervo."""
        print("\n--- LISTA DO ACERVO ---")
        if not self.__acervo:
            print("Acervo vazio.")
            return

        itens_ordenados = sorted(self.__acervo.values(), key=lambda item: item.titulo)
        hoje = datetime.now().date() # Data de hoje para checar atrasos

        for item in itens_ordenados:
            emprestado_info = ""
            if not item.disponivel:
                id_usuario, data_devolucao_limite = self.__emprestimos.get(item.titulo, (None, None))
                usuario = self.buscar_usuario(id_usuario)

                if usuario and data_devolucao_limite:
                    atrasado = "[ATRASADO!]" if hoje > data_devolucao_limite else ""
                    data_str = data_devolucao_limite.strftime('%d/%m/%Y')
                    emprestado_info = f" (para {usuario.nome} - Limite: {data_str} {atrasado})"
                else:
                    emprestado_info = " (Para Usuário Desconhecido/Erro no Rastreio)"

            print(f"- {item}{emprestado_info}")

    def listar_usuarios(self):
        """Lista todos os usuários registrados."""
        print("\n--- LISTA DE USUÁRIOS ---")
        if not self.__usuarios:
            print("Nenhum usuário registrado.")
            return
        for usuario in self.__usuarios.values():
            print(f"- {usuario}")


# --- 5. INTERFACE DE CONSOLE (ADAPTADA) ---
def exibir_menu():
    print("\n" + "="*40)
    print("      SISTEMA DE GESTÃO DA BIBLIOTECA")
    print("      (Prazo atual: 7 dias)")
    print("="*40)
    print("1. Adicionar Novo Livro")
    print("2. Adicionar Nova Revista")
    print("3. Registrar Novo Usuário")
    print("4. Emprestar Item (Livro/Revista)")
    print("5. Devolver Item (Livro/Revista)")
    print("6. Listar Acervo (Verificar Atrasos)")
    print("7. Listar Usuários")
    print("8. Sair")
    print("-" * 40)

def main_interface():
    biblioteca = Biblioteca(dias_max_emprestimos=7) # Definindo um prazo padrão

    # --- PRÉ-CARREGAR DADOS ---
    biblioteca.adicionar_item(Livro("Harry Potter e a Pedra Filosofal", "J. K. Rowling"))
    biblioteca.adicionar_item(Revista("National Geographic", "Janeiro 2025", "NatGeo Editora"))
    biblioteca.adicionar_item(Livro("Dom Quixote", "Miguel de Cervantes"))
    biblioteca.adicionar_usuario(Usuario(10, "Maria"))
    biblioteca.adicionar_usuario(Usuario(20, "João"))
    print("\n[Dados iniciais carregados para demonstração]")
    # -----------------------------------------------------------

    while True:
        exibir_menu()
        escolha = input("Escolha uma opção (1-8): ")

        if escolha == '1':
            print("\n--- ADICIONAR LIVRO ---")
            titulo = input("Título do Livro: ").strip()
            autor = input("Autor do Livro: ").strip()
            if titulo and autor:
                biblioteca.adicionar_item(Livro(titulo, autor))
            else:
                print("ERRO: Título e Autor não podem ser vazios.")

        elif escolha == '2':
            print("\n--- ADICIONAR REVISTA ---")
            titulo = input("Título da Revista: ").strip()
            edicao = input("Edição/Data da Revista: ").strip()
            editora = input("Editora/Autor: ").strip()
            if titulo and edicao and editora:
                biblioteca.adicionar_item(Revista(titulo, edicao, editora))
            else:
                print("ERRO: Todos os campos da revista devem ser preenchidos.")

        elif escolha == '3':
            print("\n--- REGISTRAR USUÁRIO ---")
            try:
                id_usuario = int(input("ID do Usuário (apenas números): ").strip())
                nome = input("Nome do Usuário: ").strip()
                if nome:
                    biblioteca.adicionar_usuario(Usuario(id_usuario, nome))
                else:
                    print("ERRO: O nome do usuário não pode ser vazio.")
            except ValueError:
                print("ERRO: ID do usuário deve ser um número inteiro.")

        elif escolha == '4':
            print("\n--- EMPRESTAR ITEM (LIVRO/REVISTA) ---")
            try:
                id_usuario = int(input("ID do Usuário que está emprestando: ").strip())
                titulo_item = input("Título do Item (Livro ou Revista) a ser emprestado: ").strip()
                biblioteca.emprestar_item(id_usuario, titulo_item)
            except ValueError:
                print("ERRO: ID do usuário deve ser um número inteiro.")

        elif escolha == '5':
            print("\n--- DEVOLVER ITEM (LIVRO/REVISTA) ---")
            try:
                id_usuario = int(input("ID do Usuário que está devolvendo: ").strip())
                titulo_item = input("Título do Item (Livro ou Revista) a ser devolvido: ").strip()
                biblioteca.devolver_item(id_usuario, titulo_item)
            except ValueError:
                print("ERRO: ID do usuário deve ser um número inteiro.")

        elif escolha == '6':
            biblioteca.listar_acervo()

        elif escolha == '7':
            biblioteca.listar_usuarios()

        elif escolha == '8':
            print("\nObrigado por usar o Sistema da Biblioteca. Saindo...")
            break

        else:
            print("Opção inválida. Por favor, escolha um número de 1 a 8.")

        input("\nPressione ENTER para continuar...")


if __name__ == '__main__':
    main_interface()

SUCESSO: Livro 'Harry Potter e a Pedra Filosofal' adicionado ao acervo.
SUCESSO: Revista 'National Geographic' adicionado ao acervo.
SUCESSO: Livro 'Dom Quixote' adicionado ao acervo.
SUCESSO: Usuário 'Maria' (ID: 10) registrado.
SUCESSO: Usuário 'João' (ID: 20) registrado.

[Dados iniciais carregados para demonstração]

      SISTEMA DE GESTÃO DA BIBLIOTECA
      (Prazo atual: 7 dias)
1. Adicionar Novo Livro
2. Adicionar Nova Revista
3. Registrar Novo Usuário
4. Emprestar Item (Livro/Revista)
5. Devolver Item (Livro/Revista)
6. Listar Acervo (Verificar Atrasos)
7. Listar Usuários
8. Sair
----------------------------------------

--- LISTA DO ACERVO ---
- [LIVRO] 'Dom Quixote' por Miguel de Cervantes | Status: Disponível
- [LIVRO] 'Harry Potter e a Pedra Filosofal' por J. K. Rowling | Status: Disponível
- [REVISTA] 'National Geographic' (Ed. Janeiro 2025) | Editora: NatGeo Editora | Status: Disponível

      SISTEMA DE GESTÃO DA BIBLIOTECA
      (Prazo atual: 7 dias)
1. Adicionar Novo 

KeyboardInterrupt: Interrupted by user